From modin main and snowpark python repo: `pip install -e .`

In [1]:
import snowflake.snowpark.modin.plugin
import modin.pandas as pd
import numpy as np
import datetime
import pandas as native_pd
from snowflake.snowpark.session import Session; session = Session.builder.create()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://snowbiz.okta.com/app/snowflake/exk8wfsfryJIn4IWZ2p7/sso/saml?SAMLRequest=jVLRctowEPwVj%2FpsSzhQiAbIUCiJMwmYYMKUN8UWRIMtuTrZBr6%2BMoZO%2BpBM3zSn3du92%2BvfHbLUKbkGoeQAtTyCHC5jlQi5G6BVNHV7yAHDZMJSJfkAHTmgu2EfWJbmdFSYd%2FnCfxccjGMbSaD1xwAVWlLFQACVLONATUyXo%2Bcn6nuEMgCujZVDF0oCwmq9G5NTjKuq8qobT%2Bkd9gkhmNxii6oh39AHifxrjVwro2KVXikHO9MnEi1M2rWERViF8EL8IWSzgq9U3hoQ0IcoCt1wvoyQM7pON1YSiozrJdeliPnq5akxANbBcjZfP8xXy58eSFVtU7bnscrywthunn3hLU9wqnbC7iiYDFC%2BF0nZ3XU7z%2BFiEbVP07C3mf9azHZld7x4vBensbov9WozX5e%2Bfh3FyHm9JurXiQYABQ9knaOxJeJ3XNJ2SS%2FyCe206Y3v9W6%2Fb5AzsTkKycyZeTVbW3wTJ0%2FtDTubY3mO%2F%2FrG%2FLDvVVvY6uNjINvBeuPnXQygcB0Tai6Fng3o4f%2FO38cfWZdjm9n9B5NQpSI%2BOlOlM2Y%2Bj6fltc4VkbjbM5TyjIl0lCSaA9iY0lRVY82ZsTdtdMERHjaq%2F1718A8%3D&RelayState=

# Example 1: Start with small dataset perform some processing, then join with large data

In [2]:
# emphemeral dataframe/lookup table 
# List of U.S. federal holidays
us_holidays = [
    ("New Year's Day", "2025-01-01"),
    ("Martin Luther King Jr. Day", "2025-01-20"),
    ("Presidents' Day", "2025-02-17"),
    ("Memorial Day", "2025-05-26"),
    ("Juneteenth National Independence Day", "2025-06-19"),
    ("Independence Day", "2025-07-04"),
    ("Labor Day", "2025-09-01"),
    ("Columbus Day", "2025-10-13"),
    ("Veterans Day", "2025-11-11"),
    ("Thanksgiving Day", "2025-11-27"),
    ("Christmas Day", "2025-12-25")
]

# Create DataFrame
df_us_holidays = pd.DataFrame(us_holidays, columns=["Holiday", "Date"])

# Convert Date column to datetime
df_us_holidays["Date"] = pd.to_datetime(df_us_holidays["Date"])

In [3]:
assert df_us_holidays.get_backend() == 'Pandas'  # with auto, we should expect this to be local

In [4]:
# Add new columns for transformations
df_us_holidays["Day_of_Week"] = df_us_holidays["Date"].dt.day_name()
df_us_holidays["Month"] = df_us_holidays["Date"].dt.month_name()

In [5]:
df_us_holidays

,Holiday,Date,Day_of_Week,Month
0,New Year's Day,2025-01-01,Wednesday,January
1,Martin Luther King Jr. Day,2025-01-20,Monday,January
2,Presidents' Day,2025-02-17,Monday,February
3,Memorial Day,2025-05-26,Monday,May
4,Juneteenth National Independence Day,2025-06-19,Thursday,June
5,Independence Day,2025-07-04,Friday,July
6,Labor Day,2025-09-01,Monday,September
7,Columbus Day,2025-10-13,Monday,October
8,Veterans Day,2025-11-11,Tuesday,November
9,Thanksgiving Day,2025-11-27,Thursday,November


In [6]:
%%time
#Note that without auto-switching, this took 2.5 min
for index, row in df_us_holidays.iterrows():
    print(f"{row['Holiday']} falls on {row['Day_of_Week']}, {row['Month']} {row['Date'].day}, {row['Date'].year}.")

New Year's Day falls on Wednesday, January 1, 2025.
Martin Luther King Jr. Day falls on Monday, January 20, 2025.
Presidents' Day falls on Monday, February 17, 2025.
Memorial Day falls on Monday, May 26, 2025.
Juneteenth National Independence Day falls on Thursday, June 19, 2025.
Independence Day falls on Friday, July 4, 2025.
Labor Day falls on Monday, September 1, 2025.
Columbus Day falls on Monday, October 13, 2025.
Veterans Day falls on Tuesday, November 11, 2025.
Thanksgiving Day falls on Thursday, November 27, 2025.
Christmas Day falls on Thursday, December 25, 2025.
CPU times: user 124 ms, sys: 7.34 ms, total: 131 ms
Wall time: 129 ms


In [7]:
pandas_df = df_us_holidays #.move_to("pandas") # remove this once we have auto-switching

In [8]:
assert pandas_df.get_backend() == 'Pandas' # With auto we expect this to be local

### Another mini example: generate synthetic data

In [9]:
%%time
# Generate 10 million transactions (This is also very slow with Snowpark pandas)
import uuid
import pandas
dates = pandas.date_range(start="2025-01-01", end="2025-12-31", freq="D")
num_transactions = 10000000
data = {
    "Transaction_ID": [str(uuid.uuid4()) for _ in range(num_transactions)],
    "Date": np.random.choice(dates, num_transactions),
    "Revenue": np.random.uniform(10, 1000, num_transactions)
}
df_transactions = pd.DataFrame(data)

CPU times: user 24 s, sys: 9.56 s, total: 33.5 s
Wall time: 33.5 s


In [10]:
assert df_transactions.get_backend() == "Pandas"

### 💡 Automatic switching speeds up loops/iterations on small data + inline creation of dataframes

## Example 2: Demonstrate that when data is prefiltered via SQL, the engine choice changes

In [11]:
# Run the following to generate a synthetic dataset with 10M rows of transactions (from 2024-2025 current date)
session.sql('''
CREATE OR REPLACE TABLE revenue_transactions (
    Transaction_ID STRING,
    Date DATE,
    Revenue FLOAT
);''').collect()
session.sql('''SET num_days = (SELECT DATEDIFF(DAY, '2024-01-01', CURRENT_DATE));''').collect()
session.sql('''INSERT INTO revenue_transactions (Transaction_ID, Date, Revenue)
SELECT
    UUID_STRING() AS Transaction_ID,
    DATEADD(DAY, UNIFORM(0, $num_days, RANDOM()), '2024-01-01') AS Date,
    UNIFORM(10, 1000, RANDOM()) AS Revenue
FROM TABLE(GENERATOR(ROWCOUNT => 10000000));
''').collect()

[Row(number of rows inserted=10000000)]

In [12]:
df_transactions = pd.read_snowflake("REVENUE_TRANSACTIONS")

In [13]:
len(df_transactions)

10000000

In [14]:
df_transactions.groupby("DATE").sum()["REVENUE"]

DATE
2024-01-01    10889216.0
2024-01-02    10670673.0
2024-01-03    10802956.0
2024-01-04    10919954.0
2024-01-05    10815703.0
                 ...    
2025-04-04    10825394.0
2025-04-05    10900718.0
2025-04-06    10897880.0
2025-04-07    10964745.0
2025-04-08    10844813.0
Name: REVENUE, Length: 464, dtype: float64

In [15]:
assert df_transactions.get_backend() == "Snowflake"

In [16]:
# Filter to records in last 7 days
df_transactions_filter = pd.read_snowflake("SELECT * FROM revenue_transactions WHERE Date >= DATEADD( 'days', -7, current_date ) and Date < current_date")

Transferring data from Snowflake to Pandas ...:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
len(df_transactions_filter)

151411

In [18]:
assert df_transactions_filter.get_backend() == "Pandas" # This should work once auto switching is on

In [19]:
df_transactions_filter.groupby("DATE").sum()["REVENUE"]

DATE
2025-04-01    10886701.0
2025-04-02    10995917.0
2025-04-03    10830577.0
2025-04-04    10825394.0
2025-04-05    10900718.0
2025-04-06    10897880.0
2025-04-07    10964745.0
Name: REVENUE, dtype: float64

### 💡 Automatic switching means that pandas work well for both small and large data

# Example 3: 

Forecast using last year's transaction data via a custom apply function

In [20]:
start_date = pd.Timestamp("2025-10-01")
end_date = pd.Timestamp("2025-10-31")

df_transactions_filtered = df_transactions[
    (df_transactions["DATE"] >= start_date - pd.Timedelta(days=365)) &
    (df_transactions["DATE"] < end_date - pd.Timedelta(days=365))
]

In [21]:
len(df_transactions_filtered)

646041

In [22]:
df_transactions_filtered_pandas = df_transactions #.move_to("pandas")

In [23]:
# Forecasting function using df.apply

def forecast_revenue(df, start_date, end_date):
    # Filter data from last year
    df_filtered = df[(df["DATE"] >= start_date - pd.Timedelta(days=365)) & (df["DATE"] < start_date)]
    
    # Append future dates to daily_avg for prediction
    future_dates = pd.date_range(start=start_date, end=end_date, freq="D")
    df_future = pd.DataFrame({"DATE": future_dates})

    # Group by DATE and calculate the mean revenue
    daily_avg = df_filtered.groupby("DATE")["REVENUE"].mean().reset_index()
    daily_avg["DATE"] = daily_avg["DATE"].astype('datetime64[ns]')
    # Merge future dates with predicted revenue, filling missing values
    df_forecast = df_future.merge(daily_avg, on="DATE", how="left")
    
    import numpy as np
    # Fill missing predicted revenue with overall mean from last year
    df_forecast["PREDICTED_REVENUE"] = np.nan
    df_forecast["PREDICTED_REVENUE"].fillna(daily_avg["REVENUE"].mean(), inplace=True)
    df_forecast["PREDICTED_REVENUE"] = df_forecast["PREDICTED_REVENUE"].astype("float")
    return df_forecast

In [24]:
# Example usage
df_forecast = forecast_revenue(df_transactions, start_date, end_date)
df_forecast

Transferring data from Snowflake to Pandas ...:   0%|          | 0/2 [00:00<?, ?it/s]

,DATE,REVENUE,PREDICTED_REVENUE
0,2025-10-01,NaN,504.867568
1,2025-10-02,NaN,504.867568
2,2025-10-03,NaN,504.867568
3,2025-10-04,NaN,504.867568
4,2025-10-05,NaN,504.867568
5,2025-10-06,NaN,504.867568
6,2025-10-07,NaN,504.867568
7,2025-10-08,NaN,504.867568
8,2025-10-09,NaN,504.867568
9,2025-10-10,NaN,504.867568


In [25]:
def adjust_for_holiday_weekend(row):
    # For national holidays, revenue down 20% since stores are closed. For weekends, revenue is up 20% due to increased activity.
    if row["DATE"].strftime('%Y-%m-%d') in list(df_us_holidays["Date"].dt.strftime('%Y-%m-%d')): 
        return row["PREDICTED_REVENUE"] * 0.8
    elif row["DATE"].weekday() == 5 or row["DATE"].weekday() == 6: #Saturday/Sundays
        return row["PREDICTED_REVENUE"] * 1.2
    return row["PREDICTED_REVENUE"]

In [26]:
df_forecast_pandas = df_forecast

In [27]:
# Adjust for holidays
df_forecast_pandas["PREDICTED_REVENUE"] = df_forecast_pandas.apply(adjust_for_holiday_weekend, axis=1)

In [28]:
df_forecast["PREDICTED_REVENUE"] = df_forecast.apply(adjust_for_holiday_weekend, axis=1)

In [29]:
type(df_forecast)

modin.pandas.dataframe.DataFrame

In [30]:
import altair as alt
alt.data_transformers.disable_max_rows()
chart_predicted = alt.Chart(df_forecast).mark_line(color='blue').encode(
    x='monthdate(DATE):T',
    y='PREDICTED_REVENUE:Q',
    tooltip=['DATE', 'PREDICTED_REVENUE']
).properties(title="Predicted Revenue")
chart_predicted

alt.Chart(...)